In [1]:
import pandas as pd
import glob

In [2]:
# get data file names
path =r'./data/rent_prices'
filenames = glob.glob(path + "/*.csv")
print(filenames)

['./data/rent_prices/2017_lloguer_preu_trim.csv', './data/rent_prices/2014_lloguer_preu_trim.csv', './data/rent_prices/2019_lloguer_preu_trim.csv', './data/rent_prices/2016_lloguer_preu_trim.csv', './data/rent_prices/2020_lloguer_preu_trim.csv', './data/rent_prices/2015_lloguer_preu_trim.csv', './data/rent_prices/2018_lloguer_preu_trim.csv']


In [3]:
# read all the *.csv inside rent_prices/
all_rent_dfs = []
for file in filenames:
    df_per_year = pd.read_csv(file)
    all_rent_dfs.append(df_per_year)
    print(f'Size of {file}: {df_per_year.shape}')

Size of ./data/rent_prices/2017_lloguer_preu_trim.csv: (584, 8)
Size of ./data/rent_prices/2014_lloguer_preu_trim.csv: (584, 8)
Size of ./data/rent_prices/2019_lloguer_preu_trim.csv: (584, 8)
Size of ./data/rent_prices/2016_lloguer_preu_trim.csv: (584, 8)
Size of ./data/rent_prices/2020_lloguer_preu_trim.csv: (292, 8)
Size of ./data/rent_prices/2015_lloguer_preu_trim.csv: (584, 8)
Size of ./data/rent_prices/2018_lloguer_preu_trim.csv: (584, 8)


In [4]:
# make a super-dataframe with all the years
rent_df = pd.concat(all_rent_dfs, axis=0, ignore_index=True)
print(rent_df.shape)
rent_df.head()

(3796, 8)


,Any,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Lloguer_mitja,Preu
0,2017,1,1,Ciutat Vella,1,el Raval,Lloguer mitjà mensual (Euros/mes),734.99
1,2017,1,1,Ciutat Vella,2,el Barri Gòtic,Lloguer mitjà mensual (Euros/mes),905.26
2,2017,1,1,Ciutat Vella,3,la Barceloneta,Lloguer mitjà mensual (Euros/mes),722.78
3,2017,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Lloguer mitjà mensual (Euros/mes),895.28
4,2017,1,2,Eixample,5,el Fort Pienc,Lloguer mitjà mensual (Euros/mes),871.08


In [5]:
# rename the columns
rent_df = rent_df.rename(columns={'Any':'year',
                   'Trimestre':'quarter', 
                   'Codi_Districte':'district_code', 
                   'Nom_Districte':'district_name', 
                   'Codi_Barri':'neighborhood_code', 
                   'Nom_Barri':'neighborhood_name', 
                   'Lloguer_mitja':'type_mean_rent', 
                   'Preu':'price'})
rent_df.head()

,year,quarter,district_code,district_name,neighborhood_code,neighborhood_name,type_mean_rent,price
0,2017,1,1,Ciutat Vella,1,el Raval,Lloguer mitjà mensual (Euros/mes),734.99
1,2017,1,1,Ciutat Vella,2,el Barri Gòtic,Lloguer mitjà mensual (Euros/mes),905.26
2,2017,1,1,Ciutat Vella,3,la Barceloneta,Lloguer mitjà mensual (Euros/mes),722.78
3,2017,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Lloguer mitjà mensual (Euros/mes),895.28
4,2017,1,2,Eixample,5,el Fort Pienc,Lloguer mitjà mensual (Euros/mes),871.08


In [6]:
rent_df = rent_df.assign(type_mean_rent= lambda df: 
                         df['type_mean_rent'].map({'Lloguer mitjà mensual (Euros/mes)':'Euros/month','Lloguer mitjà per superfície (Euros/m2 mes)':'Euros/m2 month'}))
rent_df.head()

,year,quarter,district_code,district_name,neighborhood_code,neighborhood_name,type_mean_rent,price
0,2017,1,1,Ciutat Vella,1,el Raval,Euros/month,734.99
1,2017,1,1,Ciutat Vella,2,el Barri Gòtic,Euros/month,905.26
2,2017,1,1,Ciutat Vella,3,la Barceloneta,Euros/month,722.78
3,2017,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Euros/month,895.28
4,2017,1,2,Eixample,5,el Fort Pienc,Euros/month,871.08


In [7]:
def make_period(row):
    return f"{row['year']}Q{row['quarter']}"

In [8]:
rent_df['period'] = rent_df.apply(make_period, axis=1)
rent_df['period'] = pd.to_datetime(rent_df['period'])
rent_df['period']

0      2017-01-01
1      2017-01-01
2      2017-01-01
3      2017-01-01
4      2017-01-01
          ...    
3791   2018-10-01
3792   2018-10-01
3793   2018-10-01
3794   2018-10-01
3795   2018-10-01
Name: period, Length: 3796, dtype: datetime64[ns]

In [9]:
groupby_object = rent_df.groupby(by='type_mean_rent')
cleaned_rent_df = groupby_object.get_group('Euros/month')
print(cleaned_rent_df.shape)
cleaned_rent_df.head()

(1898, 9)


,year,quarter,district_code,district_name,neighborhood_code,neighborhood_name,type_mean_rent,price,period
0,2017,1,1,Ciutat Vella,1,el Raval,Euros/month,734.99,2017-01-01
1,2017,1,1,Ciutat Vella,2,el Barri Gòtic,Euros/month,905.26,2017-01-01
2,2017,1,1,Ciutat Vella,3,la Barceloneta,Euros/month,722.78,2017-01-01
3,2017,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Euros/month,895.28,2017-01-01
4,2017,1,2,Eixample,5,el Fort Pienc,Euros/month,871.08,2017-01-01


In [10]:
cleaned_rent_df = cleaned_rent_df.rename(columns={'price':'price_month(€)'})
cleaned_rent_df.head()

,year,quarter,district_code,district_name,neighborhood_code,neighborhood_name,type_mean_rent,price_month(€),period
0,2017,1,1,Ciutat Vella,1,el Raval,Euros/month,734.99,2017-01-01
1,2017,1,1,Ciutat Vella,2,el Barri Gòtic,Euros/month,905.26,2017-01-01
2,2017,1,1,Ciutat Vella,3,la Barceloneta,Euros/month,722.78,2017-01-01
3,2017,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Euros/month,895.28,2017-01-01
4,2017,1,2,Eixample,5,el Fort Pienc,Euros/month,871.08,2017-01-01


In [11]:
cleaned_rent_df.drop('type_mean_rent', axis=1, inplace=True)
cleaned_rent_df.head()

,year,quarter,district_code,district_name,neighborhood_code,neighborhood_name,price_month(€),period
0,2017,1,1,Ciutat Vella,1,el Raval,734.99,2017-01-01
1,2017,1,1,Ciutat Vella,2,el Barri Gòtic,905.26,2017-01-01
2,2017,1,1,Ciutat Vella,3,la Barceloneta,722.78,2017-01-01
3,2017,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",895.28,2017-01-01
4,2017,1,2,Eixample,5,el Fort Pienc,871.08,2017-01-01


In [12]:
rent_m2_month_df = groupby_object.get_group('Euros/m2 month')
#rent_m2_month_df = rent_m2_month_df.set_index(['neighborhood_code', 'period'])
rent_m2_month_df.head()

,year,quarter,district_code,district_name,neighborhood_code,neighborhood_name,type_mean_rent,price,period
292,2017,1,1,Ciutat Vella,1,el Raval,Euros/m2 month,13.50,2017-01-01
293,2017,1,1,Ciutat Vella,2,el Barri Gòtic,Euros/m2 month,14.14,2017-01-01
294,2017,1,1,Ciutat Vella,3,la Barceloneta,Euros/m2 month,19.46,2017-01-01
295,2017,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Euros/m2 month,15.01,2017-01-01
296,2017,1,2,Eixample,5,el Fort Pienc,Euros/m2 month,12.77,2017-01-01


In [13]:
cleaned_rent_df = cleaned_rent_df.merge(rent_m2_month_df, how='left', on=['neighborhood_code', 'period'], suffixes=('', '_repeated'))
cleaned_rent_df.head()

,year,quarter,district_code,district_name,neighborhood_code,neighborhood_name,price_month(€),period,year_repeated,quarter_repeated,district_code_repeated,district_name_repeated,neighborhood_name_repeated,type_mean_rent,price
0,2017,1,1,Ciutat Vella,1,el Raval,734.99,2017-01-01,2017,1,1,Ciutat Vella,el Raval,Euros/m2 month,13.50
1,2017,1,1,Ciutat Vella,2,el Barri Gòtic,905.26,2017-01-01,2017,1,1,Ciutat Vella,el Barri Gòtic,Euros/m2 month,14.14
2,2017,1,1,Ciutat Vella,3,la Barceloneta,722.78,2017-01-01,2017,1,1,Ciutat Vella,la Barceloneta,Euros/m2 month,19.46
3,2017,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",895.28,2017-01-01,2017,1,1,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",Euros/m2 month,15.01
4,2017,1,2,Eixample,5,el Fort Pienc,871.08,2017-01-01,2017,1,2,Eixample,el Fort Pienc,Euros/m2 month,12.77


In [14]:
cleaned_rent_df = cleaned_rent_df.drop(list(cleaned_rent_df.filter(regex='_repeated')), axis=1)
cleaned_rent_df.head()

,year,quarter,district_code,district_name,neighborhood_code,neighborhood_name,price_month(€),period,type_mean_rent,price
0,2017,1,1,Ciutat Vella,1,el Raval,734.99,2017-01-01,Euros/m2 month,13.50
1,2017,1,1,Ciutat Vella,2,el Barri Gòtic,905.26,2017-01-01,Euros/m2 month,14.14
2,2017,1,1,Ciutat Vella,3,la Barceloneta,722.78,2017-01-01,Euros/m2 month,19.46
3,2017,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",895.28,2017-01-01,Euros/m2 month,15.01
4,2017,1,2,Eixample,5,el Fort Pienc,871.08,2017-01-01,Euros/m2 month,12.77


In [15]:
cleaned_rent_df.drop('type_mean_rent', axis=1, inplace=True)
cleaned_rent_df.head()

,year,quarter,district_code,district_name,neighborhood_code,neighborhood_name,price_month(€),period,price
0,2017,1,1,Ciutat Vella,1,el Raval,734.99,2017-01-01,13.50
1,2017,1,1,Ciutat Vella,2,el Barri Gòtic,905.26,2017-01-01,14.14
2,2017,1,1,Ciutat Vella,3,la Barceloneta,722.78,2017-01-01,19.46
3,2017,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",895.28,2017-01-01,15.01
4,2017,1,2,Eixample,5,el Fort Pienc,871.08,2017-01-01,12.77


In [16]:
cleaned_rent_df = cleaned_rent_df.rename(columns={'price':'price_m2_month(€)'})
cleaned_rent_df.head()

,year,quarter,district_code,district_name,neighborhood_code,neighborhood_name,price_month(€),period,price_m2_month(€)
0,2017,1,1,Ciutat Vella,1,el Raval,734.99,2017-01-01,13.50
1,2017,1,1,Ciutat Vella,2,el Barri Gòtic,905.26,2017-01-01,14.14
2,2017,1,1,Ciutat Vella,3,la Barceloneta,722.78,2017-01-01,19.46
3,2017,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",895.28,2017-01-01,15.01
4,2017,1,2,Eixample,5,el Fort Pienc,871.08,2017-01-01,12.77


In [17]:
cleaned_rent_df = cleaned_rent_df.reindex(columns=['year', 'quarter', 'period', 'district_code', 'district_name', 'neighborhood_code', 'neighborhood_name', 'price_month(€)', 'price_m2_month(€)'])
cleaned_rent_df.head()

,year,quarter,period,district_code,district_name,neighborhood_code,neighborhood_name,price_month(€),price_m2_month(€)
0,2017,1,2017-01-01,1,Ciutat Vella,1,el Raval,734.99,13.50
1,2017,1,2017-01-01,1,Ciutat Vella,2,el Barri Gòtic,905.26,14.14
2,2017,1,2017-01-01,1,Ciutat Vella,3,la Barceloneta,722.78,19.46
3,2017,1,2017-01-01,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",895.28,15.01
4,2017,1,2017-01-01,2,Eixample,5,el Fort Pienc,871.08,12.77


In [19]:
cleaned_rent_df.to_csv('./data/cleaned/rent_prices.csv', index=False)